In [4]:
from selenium import webdriver
import sys
import time
#sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import pandas as pd


### setup
antes de correr debes descargar el chrome driver acorde a tu maquina
https://sites.google.com/chromium.org/driver/downloads


### helper functions

estas funciones son secundarias y nos ayudan a correr el main script del scrapper 

In [5]:
def init_scrapper(driverpath,keyword,headless = False ):
    """"initialize the glassdor scrapper given a job keyword"""
    # Initializing the webdriver
    options = webdriver.ChromeOptions()

    # if you'd like to scrape without a new Chrome window every time.
    if headless: options.add_argument('headless')

    # Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path=driverpath, options=options)
    url = "https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword="+keyword+"&sc.keyword="+keyword+"&locT=&locId=&jobType="
    driver.get(url)

    return driver


def get_companyname_rating(company_name):
    """gets the company name and (if exist) gets the company rating too"""
    splited = company_name.split("\n")
    # company name comes with rating 
    if len(splited) > 1:
        return splited[0],  splited[1]
    # just company name
    else:
        return company_name, -1


def get_company_overview_data(Company_Overview):
    """"gets the company overview data (dont the name of the func speaks for itself?)"""
    Company_Overview_dict = {"size":"",
                            "founded":"",
                            "type":"",
                            "industry":"",
                            "sector":"",
                            "revenue":"",}

    for i, key in enumerate(Company_Overview_dict):
        try: 
            Company_Overview_dict[key] = Company_Overview[i].text
        except:
            print(f"{key} not found")
            Company_Overview_dict[key] = -1
    return Company_Overview_dict


def response_to_dataframe(list_of_dicts,filename):
  """cast list of dict into df, if filename is specified exports the csv """
  df = pd.DataFrame(list_of_dicts)
  if filename:
    df.to_csv(f'./{filename}.csv')
  return df



In [6]:
path = "/opt/homebrew/bin/chromedriver"
keyword = "data science"
driver = init_scrapper(path,keyword)

/var/folders/n2/np9y87ks47j0gqm4v82gvddh0000gn/T/ipykernel_40998/4098912020.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=driverpath, options=options)


In [12]:
jobs = driver.find_elements(By.CLASS_NAME,'react-job-listing')
jobs

[<selenium.webdriver.remote.webelement.WebElement (session="589c8f4d702b9e022f64627577024318", element="6eb49b14-99ee-4e91-b582-0ca5cfaf2a55")>,
 <selenium.webdriver.remote.webelement.WebElement (session="589c8f4d702b9e022f64627577024318", element="1ae59a2d-6960-4b11-ae6c-cbb7570bf684")>,
 <selenium.webdriver.remote.webelement.WebElement (session="589c8f4d702b9e022f64627577024318", element="1f7cf397-a05b-42eb-a88a-713a182c0d86")>,
 <selenium.webdriver.remote.webelement.WebElement (session="589c8f4d702b9e022f64627577024318", element="cfabe5e0-a208-4600-aaa3-efaa53ed2419")>,
 <selenium.webdriver.remote.webelement.WebElement (session="589c8f4d702b9e022f64627577024318", element="8c4462dd-13e3-4727-bde4-4c1226276f1d")>,
 <selenium.webdriver.remote.webelement.WebElement (session="589c8f4d702b9e022f64627577024318", element="4e315d8a-9026-4dbc-b9ed-08ab783420c5")>,
 <selenium.webdriver.remote.webelement.WebElement (session="589c8f4d702b9e022f64627577024318", element="6cc8ee39-1087-4fd0-8474-50

### SCRAPPER

In [4]:



def get_jobs(keyword, num_jobs, verbose, path, slp_time, failsafe = False):
    
    """Gathers jobs as a dataframe, scraped from Glassdoor"""
    driver = init_scrapper(path,keyword)
    jobs = []
    #Let the page load. Change this number based on your internet speed.
    time.sleep(2*slp_time)
    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Going through each job in this page
        job_buttons = driver.find_elements(By.CLASS_NAME, "react-job-listing")  #react-job-listing for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:  

            print(f"Progress:  {len(jobs)}/{ num_jobs}")
            if len(jobs) >= num_jobs:
                break
            job_button.click()  #You might 
            time.sleep(slp_time)
            # Test for the "Sign Up" prompt and get rid of it.
            # close modal (just appear on the first pass)
            if len(jobs) == 0:
                try:
                    driver.find_element(By.XPATH, '//*[@id="JAModal"]/div/div[2]/span').click()
                    print("modal cerrado con exito")
                except NoSuchElementException:
                    print("no hay modal xd")
                    
            collected_successfully = False
            while not collected_successfully:
                try:

                    company_name_ = driver.find_element(By.CLASS_NAME, 'e1tk4kwz5').text
                    #sometimes company name comes with the rating, if its the case get_companyname_rating handles it
                    company_name, rating = get_companyname_rating(company_name_) 
                    location = driver.find_element(By.CLASS_NAME,'e1tk4kwz1').text
                    job_title = driver.find_element(By.CLASS_NAME,'e1tk4kwz2').text
                    job_description = driver.find_element(By.XPATH,'.//div[@class="jobDescriptionContent desc"]').text
                    salary_estimate = driver.find_element(By.CLASS_NAME,'e2u4hf18').text
                    collected_successfully = True
                    print("job info collected successfully :)")
                except:
                    time.sleep(slp_time)
                    print("job info error, trying again...")
                    break

            job_data_dict = {"Job Title" : job_title,
                            "Company Name" : company_name,
                            "Location" : location,
                            "Salary Estimate" : salary_estimate,
                            "Rating" : rating,
                            "Job Description" : job_description,
                        }

            #Printing for debugging
            if verbose:
                print(job_data_dict)

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                Company_Overview = driver.find_elements(By.CLASS_NAME,'e1pvx6aw0')
                company_overview_dict = get_company_overview_data(Company_Overview)

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                company_overview_dict = {"size":-1,
                                        "founded":-1,
                                        "type":-1,
                                        "industry":-1,
                                        "sector":-1,
                                        "revenue":-1}
                
            if verbose:
                print(company_overview_dict)
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
            
            job_data_dict.update(company_overview_dict)

            jobs.append(job_data_dict)
            if failsafe: response_to_dataframe(jobs,"jobs_df")

            #add job to jobs
            
            
        #Clicking on the "next page" button
        try:
            driver.find_element(By.CLASS_NAME,'nextButton').click()
            time.sleep(slp_time)
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break
        
    driver.close()
    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [6]:
# https://stackoverflow.com/questions/60362018/macos-catalinav-10-15-3-error-chromedriver-cannot-be-opened-because-the-de
path = "/opt/homebrew/bin/chromedriver"
keyword = "data science"
num_jobs = 500
slp_time = 3
verbose = True
failsafe = True

df = get_jobs(keyword, num_jobs, verbose, path, slp_time,failsafe)
df

/var/folders/n2/np9y87ks47j0gqm4v82gvddh0000gn/T/ipykernel_12275/1250913795.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=driverpath, options=options)


Progress:  0/500
modal cerrado con exito
job info collected successfully :)
{'Job Title': 'Data Scientist', 'Company Name': 'Agama Solutions', 'Location': 'San Francisco, CA', 'Salary Estimate': '$75.00 /hr (est.)', 'Rating': '3.8', 'Job Description': 'Position Title: Data Scientist\n\nLocation: Bay Area –CA (100% Remote)\nDuration: 12+ months\nRate $75/hr on W2\nVendor Gspann\nClient Kohl'}
revenue not found
{'size': '51 to 200 Employees', 'founded': 'Contract', 'type': 'Computer Hardware Development', 'industry': 'Information Technology', 'sector': '$10 to $25 million (USD)', 'revenue': -1}
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Progress:  1/500
job info error, trying again...
{'Job Title': 'Entry Level Data Analyst (Part-time)', 'Company Name': 'CORMAC', 'Location': 'Remote', 'Salary Estimate': '$75.00 /hr (est.)', 'Rating': '3.4', 'Job Description': "HUB Zone residence is a requirement for this job.\n\nPlease check if you qualify for this position at the link below.\n

,Job Title,Company Name,Location,Salary Estimate,Rating,Job Description,size,founded,type,industry,sector,revenue
0,Data Scientist,Agama Solutions,"San Francisco, CA",$75.00 /hr (est.),3.8,Position Title: Data Scientist\n\nLocation: Ba...,51 to 200 Employees,Contract,Computer Hardware Development,Information Technology,$10 to $25 million (USD),-1
1,Entry Level Data Analyst (Part-time),CORMAC,Remote,$75.00 /hr (est.),3.4,HUB Zone residence is a requirement for this j...,Unknown,2002,Company - Private,Software Development,Information Technology,Less than $1 million (USD)
2,"Front End Engineer, MADS- Measurement and Data...",Amazon.com Services LLC,"Boulder, CO","$93,800 /yr (est.)",3.8,Professional non-internship experience with fr...,10000+ Employees,1994,Company - Public,Internet & Web Services,Information Technology,$10+ billion (USD)
3,Jr. Data Scientist,Net2Aspire LLC,Remote,"$82,500 /yr (est.)",-1, Apply Statistical and Machine Learning metho...,-1,-1,-1,-1,-1,-1
4,Healthcare Data Scientist,Humbi Health LLC,Remote,"$175,000 /yr (est.)",-1,Humbi Health is looking for highly analytical ...,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...
495,STATISTICAL ASSISTANT,UTOFUN,Manhattan,"$37,070 /yr (est.)",4.1,Overview\nThe UTOFUN is the leading internet s...,1 to 50 Employees,Company - Private,Unknown / Non-Applicable,-1,-1,-1
496,2022 Graduate Fellow Data Science Co-op,Merck,"North Wales, PA","$83,375 /yr (est.)",4.1,Job Description\nNew hires in office-based rol...,10000+ Employees,1891,Company - Public,Biotech & Pharmaceuticals,Pharmaceutical & Biotechnology,$10+ billion (USD)
497,Student Employee - Data Science,Electric Power Research Institute,"Charlotte, NC","$75,720 /yr (est.)",5.0,Job Title:\nStudent Employee - Data Science\nL...,Unknown,Company - Private,Unknown / Non-Applicable,-1,-1,-1
498,Data Analyst,Hulu,"Santa Monica, CA","$87,765 /yr (est.)",3.9,We are seeking a Data Analyst who will focus o...,1001 to 5000 Employees,2007,Company - Private,Broadcast Media,Media & Communication,$1 to $2 billion (USD)


In [7]:
response_to_dataframe(df,"Glassdoor_jobs")

,Job Title,Company Name,Location,Salary Estimate,Rating,Job Description,size,founded,type,industry,sector,revenue
0,Data Scientist,Agama Solutions,"San Francisco, CA",$75.00 /hr (est.),3.8,Position Title: Data Scientist\n\nLocation: Ba...,51 to 200 Employees,Contract,Computer Hardware Development,Information Technology,$10 to $25 million (USD),-1
1,Entry Level Data Analyst (Part-time),CORMAC,Remote,$75.00 /hr (est.),3.4,HUB Zone residence is a requirement for this j...,Unknown,2002,Company - Private,Software Development,Information Technology,Less than $1 million (USD)
2,"Front End Engineer, MADS- Measurement and Data...",Amazon.com Services LLC,"Boulder, CO","$93,800 /yr (est.)",3.8,Professional non-internship experience with fr...,10000+ Employees,1994,Company - Public,Internet & Web Services,Information Technology,$10+ billion (USD)
3,Jr. Data Scientist,Net2Aspire LLC,Remote,"$82,500 /yr (est.)",-1, Apply Statistical and Machine Learning metho...,-1,-1,-1,-1,-1,-1
4,Healthcare Data Scientist,Humbi Health LLC,Remote,"$175,000 /yr (est.)",-1,Humbi Health is looking for highly analytical ...,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...
495,STATISTICAL ASSISTANT,UTOFUN,Manhattan,"$37,070 /yr (est.)",4.1,Overview\nThe UTOFUN is the leading internet s...,1 to 50 Employees,Company - Private,Unknown / Non-Applicable,-1,-1,-1
496,2022 Graduate Fellow Data Science Co-op,Merck,"North Wales, PA","$83,375 /yr (est.)",4.1,Job Description\nNew hires in office-based rol...,10000+ Employees,1891,Company - Public,Biotech & Pharmaceuticals,Pharmaceutical & Biotechnology,$10+ billion (USD)
497,Student Employee - Data Science,Electric Power Research Institute,"Charlotte, NC","$75,720 /yr (est.)",5.0,Job Title:\nStudent Employee - Data Science\nL...,Unknown,Company - Private,Unknown / Non-Applicable,-1,-1,-1
498,Data Analyst,Hulu,"Santa Monica, CA","$87,765 /yr (est.)",3.9,We are seeking a Data Analyst who will focus o...,1001 to 5000 Employees,2007,Company - Private,Broadcast Media,Media & Communication,$1 to $2 billion (USD)


In [9]:
df.to_csv('jobs.csv')